In [1]:
import os
import torch.autograd
import torch.nn as nn



In [2]:
#from Model.CVAE_PG_GAN import Gen_Block
# z=torch.randn(64,100)
# y=torch.randn(64,10)
# z0=torch.concat([z,y],dim=1)

# linear=nn.Linear(110,3*16*16)
#gen_block=Gen_Block(3,32,3,1,1,100,10)
#print(gen_block)
# x=linear(z0)
# print(x.shape)
# x=x.view(x.size(0),-1,16,16)
# print(x.shape)
# x1=gen_block(x,z0)
# print(x1.shape)


In [3]:
import torch
from Model.CVAE_GAN_Deep import CVAE as CVAE_DEEP
from Model.CVAE_GAN import CVAE
y=torch.randn(64,3,64,64).to("cuda")
cvae_deep = CVAE_DEEP(nz=100,
			 imSize=64,
             block_num=5,
             in_channel=3,
			 fSize=64,
             numLabels=10,
			 device='cuda')
print(cvae_deep)
cvae_deep.to("cuda")
cvae=cvae = CVAE(nz=100,
			 imSize=64,
			 enc_self_attn=True,
			 dec_self_attn=True,
			 g_spectral_norm=True,
			 CBN=True,
			 fSize=64,
             numLabels=10,
			 device='cuda')
print(cvae)
cvae.to('cuda')
mu1, log_var1, y1 =cvae.encode(y)
z1 = cvae.re_param(mu1, log_var1)
reconstruction1 = cvae.decode(y1,z1)
print(reconstruction1.shape)

mu2, log_var2, y2 =cvae_deep.encode(y)
z2 = cvae_deep.re_param(mu2, log_var2)
reconstruction2 = cvae_deep.decode(y2,z2)
print(reconstruction2.shape)





CVAE(
  (encLogVar): Linear(in_features=4096, out_features=100, bias=True)
  (encMu): Linear(in_features=4096, out_features=100, bias=True)
  (encY): Linear(in_features=4096, out_features=10, bias=True)
  (enc_layers): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (2): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (3): Conv2d(256, 512, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (4): Conv2d(512, 1024, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  )
  (enc_attn_block): Self_Attn(
    (conv1x1_theta): Conv2d(1024, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv1x1_phi): Conv2d(1024, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv1x1_g): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv1x1_attn): Conv2d(512, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (maxpool): 

e:\Project\ZLTProgram\GTSRB\Model\CVAE_GAN.py:159: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(self.encY(x.detach()))
e:\Project\ZLTProgram\GTSRB\Model\CVAE_GAN_Deep.py:133: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(self.encY(x.detach()))


In [4]:
# print(z.shape)
# z=torch.cat([z,y],dim=1)
# z.shape
# z0=z
# import torch.nn.functional as F
# z=F.relu(cvae_deep.dec_fc(z))
# z=z.view(z.size(0), -1, 64, 64)
# z.shape

In [5]:
# import torch.nn.functional as F
# mu, log_var, y =cvae_deep.encode(y)
# z = cvae_deep.re_param(mu, log_var)
# print(z.shape)
# z=torch.cat([z,y],dim=1)
# z0=z
# z=F.relu(cvae_deep.dec_fc(z))
# z=z.view(z.size(0), -1, 64, 64)
# z=cvae_deep.dec_blocks[0](z,z0)
# print(z.shape)

In [7]:
from Model.CVAE_PG_GAN import DISCRIMINATOR
discriminator= DISCRIMINATOR(64,4,32,1,'cuda')
print(discriminator)
discriminator.to("cuda")
dis=discriminator(reconstruction2)
print(dis)


DISCRIMINATOR(
  (dis_blocks): ModuleList(
    (0): Dis_Block(
      (layer1): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
      )
      (layer2): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
      )
    )
    (1): Dis_Block(
      (layer1): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
      )
      (layer2): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
      )
    )
    (2): Dis_Block(
      (layer1): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
      )
      (layer2): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), p